# Hypothesis testing
Elements of Data Science

## Hypothesis Testing Learning Goals
Develop and test an hypothesis
- Hypothesis
    - testable hypothesis
    - statistic
- Simulation: Sample the distribution
    - Repeat and collect outcomes
    - Iteration: 
        `for i in np.arange(samples)`
- Examine resulting distribution of outcomes
    - Probability distribution
    - Uncertainty
- p-test

In [ ]:
import numpy as np
from datascience import *

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Fix for datascience plots
import collections as collections
import collections.abc as abc
collections.Iterable = abc.Iterable

## A digression -- python formatting

You know Python has a few formatting rules. Indentation is critical for defining structure in loops, functions, and conditional statements, for example. Beyond that, you can pretty much do as you like. The following formatting conventions, however, will make your code more readable.

Black is a Python tool for checking formatting. You can actually install and run it to check your code, or you can simply paste some code you have written into the [Black Playground](https://ryanking13.github.io/black-playground-pyodide/) to see the suggested format.

In [ ]:
# First Example: Paste into the playground. Note: the code will run as written

a=2
b=3
c=a+b
print( c)

In [ ]:
# Paste the foratted code below:
# First Example: Paste into the playground. Note: the code will run as written


a = 2
b = 3
c = a + b
print(c)


In [ ]:
# Second example: Again, the code as written will run without error. But long lines are hard to read.
# Create a list
CST_departments = ['Earth & Environmental Science','Biology','Chemistry','Physics','Computer & Information Sciences','Mathematics']

# Now loop over each element and print it.
for dept in CST_departments:
    print(dept)

In [ ]:
# Paste the foratted code below:



In [ ]:
# Formatting complex statements
from datascience import *
data = 'http://www2.census.gov/programs-surveys/popest/datasets/2010-2020/national/asrh/nc-est2020-agesex-res.csv'
full_census_table = Table.read_table(data)
partial_census_table = full_census_table.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2020')
partial_census_table = partial_census_table.relabeled('SEX', 'GENDER').relabeled('POPESTIMATE2010', '2010').relabeled('POPESTIMATE2020', '2020')
partial_census_table.show(3)

In [ ]:
# Paste the foratted code below:



Black makes some opinionated choices. You do not have to follow all of its formatting suggestions, but you should pick a consistant way to format your code and Black uses what many pythonistas consider to be best practice.

## End of digression...

## Olympics Review

In [ ]:
datafile = "/home/jovyan/datascience/Spring 2024/Mini Project I/Olympic_Data/winter_athletes.csv"
athletes = Table.read_table(datafile).sort("Year",descending=True).where("Season","Winter")
age_athletes = athletes.where("Age",are.below(99)).where("Age",are.above(0))

5. What are the top ten countries in number of Gold, silver, bronze medals, and total medals? You should have four sets of top ten countries for each of the scenarios. Generate the **five number summary** for each medal type for all countries.
Hint: `.where("Medal",are.not_equal_to("nan"))` to get only medal winners. Consider how to create a column for the sum of the three medal categories.

In [ ]:
medals = athletes.where("Medal", are.not_equal_to("nan")).select("Team", "Medal").pivot("Medal", "Team")
medals.show(5)

In [ ]:
total_medals = medals.column("Gold") + medals.column("Silver") + medals.column("Bronze")
medals = medals.with_column("Total Medals", total_medals)
medals.show(5)

In [ ]:
# Top ten gold
medals.sort("Gold", descending=True).select("Team", "Gold").take(np.arange(10))

7. Which sports (top 5) awarded the most medals in Lake Placid, New York (1980,
https://www.lakeplacid.com/do/activities/olympic-sites ).

In [ ]:
# Sports that awarded the most medals in 1980
medals_sport = athletes.where("Medal", are.not_equal_to("nan"))
medals_sport.where("Year", 1980).group("Sport").sort("count", descending=True).take(np.arange(5))

12. Plot the yearly trend in number of sports. Think of a strategy to code this. What is the trend? 

In [ ]:
athletes.group(["Year", "Event"]).group("Year")

In [ ]:
athletes.group(["Year", "Event"]).group("Year").relabel("count", "Number of Events").plot("Year", "Number of Events")

13. Plot an overlay of gold, silver, and bronze medals as a function of year on the same plot excluding hockey. What is the trend? Are the medals awarded at a similar rate?

In [ ]:
medals_per_year = athletes.where("Event", are.not_containing("Hockey")).pivot("Medal", "Year").drop("nan")
medals_per_year.show(3)

In [ ]:
medals_per_year.drop("Total Medals").plot("Year")
ax = plt.gca()
ax.set_ylabel("Medals per Year");

## LAB 06 TIPS

In [ ]:
modifier = 11
num_observations = 7

def simulate_observations(modifier, num_observations):
    """Produces an array of 7 simulated modified die rolls"""
    ...

observations = ...
observations

In [ ]:
# How to make an empty array and append values
test = make_array()
np.append(test, 5)
test

# Back to Groundhogs
![](https://i.natgeofe.com/n/fb6f0b10-50e0-4857-9246-012bbef02614/groundhog.jpg?w=1200)

## Hypothesis Testing

#### Ground Hog


In [ ]:
groundhogdata = Table.read_table('../../Lab05/GroundHogData/summarizedGroundhogData_20210326.csv')
groundhogdata

In [ ]:
yes_late = groundhogdata.where('shadowPres', 'yes').where('earlyOrLate', 'late')
no_early = groundhogdata.where('shadowPres', 'no').where('earlyOrLate', 'early')

correct_tbl = yes_late.append(no_early)

In [ ]:
num_correct = correct_tbl.num_rows / groundhogdata.num_rows * 100
print(f"The groundhogs were correct {num_correct:0.1f} percent of the time.")

## Is 51.1% is correct "significantly" better than random guessing?

To answer this question we need to simulate guessing randomly 520 times (the size of our data set) and see what fractionof the time we do this much better than 50-50 by guessing.

In [ ]:
shadow_options = make_array('yes', 'no')
spring_options = make_array('late', 'early')
num_observations = groundhogdata.num_rows
num_simulations = 1500
num_observations

In [ ]:
# Simulate by guessing -- this is a single trial.

right = 0 
wrong = 0

for obs in range(num_observations):
    shadow = np.random.choice(shadow_options)
    spring = np.random.choice(spring_options)
    
    if shadow == 'yes' and spring == 'late':
        right += 1
    elif shadow == 'no' and spring == 'early':
        right += 1
    else:
        wrong += 1

simulated_num_correct = right / num_simulations
simulated_num_correct

In [ ]:
# Create a function so we can generate many trials.
def sim_ground(repeats):
    correct_obs = []
    for i in np.arange(repeats):
        right = 0 
        wrong = 0
        for obs in range(num_observations):
            shadow = np.random.choice(shadow_options)
            spring = np.random.choice(spring_options)
            if shadow == 'yes' and spring == 'late':
                right += 1
            elif shadow == 'no' and spring == 'early':
                right += 1
            else:
                wrong += 1
        simulated_num_correct = right / num_observations
        correct_obs.append(simulated_num_correct)
    return correct_obs        

In [ ]:
# Plot the histogram of 1500 trials
plt.hist(sim_ground(num_observations),bins=np.arange(0.4,0.6,.01),color = "skyblue", ec="red")
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground.png')
plt.show()

In [ ]:
hogsimdata = Table().with_columns("Correct",sim_ground(num_simulations))
hogsimdata

In [ ]:
# The red dot shows the groundhog prediction accuracy
hogsimdata.hist(0,bins=np.arange(0.4,0.6,.01))
plt.scatter(num_correct/100, 0, color='red', s=200);
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground_correct.png')
plt.show()

In [ ]:
# What is the probabilty of doing better than the groundhogs with random guessing?
np.count_nonzero(hogsimdata.column('Correct') >= num_correct/100) / num_simulations

#### Essex Ed

In [ ]:
groundhogdata.where('hogID',"EE")

In [ ]:
correct_tbl.where('hogID',"EE").sort('year')

In [ ]:
num_correct = correct_tbl.where('hogID',"EE").num_rows / groundhogdata.where('hogID',"EE").num_rows * 100
num_correct

In [ ]:
hogsimdata.hist(0,bins=np.arange(0.4,0.6,.01))
plt.scatter(num_correct/100, 0, color='red', s=200);
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground_correct.png')
plt.show()

In [ ]:
np.count_nonzero(hogsimdata.column('Correct') >= num_correct/100) / num_simulations

In [ ]:
num_observations = 7
hogsimdata = Table().with_columns("Correct",sim_ground(num_simulations))
hogsimdata

In [ ]:
#hogsimdata.hist(0,bins=np.arange(0.1,0.9,.01))
hogsimdata.hist(bins=15)
plt.scatter(num_correct/100, 0, color='red', s=200);
plt.title('Simulated Groundhog outcomes')
plt.savefig('sim_ground_correct.png')
plt.show()

In [ ]:
np.count_nonzero(hogsimdata.column('Correct') >= num_correct/100) / num_simulations

# Acknowledgement in the paper
This study was the result of a truly laboratory-driven effort that started over speculation on a Friday at the campus pub and has led to a cumulative effort of students in the Community Ecology and Energetics Laboratory at Lakehead University over several years. 